<a href="https://colab.research.google.com/github/AnatoliChe/clbench/blob/main/bench1000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!sudo apt purge *nvidia* -y
!sudo apt install nvidia-driver-530 -y

Streaming output truncated to the last 5000 lines.
Package 'linux-objects-nvidia-470-5.15.0-1050-intel-iotg' is not installed, so not removed
Package 'linux-objects-nvidia-470-5.15.0-1050-oracle' is not installed, so not removed
Package 'linux-objects-nvidia-470-5.15.0-1051-aws' is not installed, so not removed
Package 'linux-objects-nvidia-470-5.15.0-1051-azure' is not installed, so not removed
Package 'linux-objects-nvidia-470-5.15.0-1051-gcp' is not installed, so not removed
Package 'linux-objects-nvidia-470-5.15.0-1051-intel-iotg' is not installed, so not removed
Package 'linux-objects-nvidia-470-5.15.0-1051-oracle' is not installed, so not removed
Package 'linux-objects-nvidia-470-5.15.0-1052-aws' is not installed, so not removed
Package 'linux-objects-nvidia-470-5.15.0-1052-azure' is not installed, so not removed
Package 'linux-objects-nvidia-470-5.15.0-1052-gcp' is not installed, so not removed
Package 'linux-objects-nvidia-470-5.15.0-1052-intel-iotg' is not installed, so not re

In [2]:
!apt install pocl-opencl-icd
!pip install numpy
!pip install pyopencl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  ocl-icd-opencl-dev opencl-c-headers opencl-clhpp-headers
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  binfmt-support clang-11 libclang-common-11-dev libclang-cpp11 libclang1-11 libffi-dev libllvm11
  libpfm4 libpocl2 libpocl2-common libz3-4 libz3-dev llvm-11 llvm-11-dev llvm-11-linker-tools
  llvm-11-runtime llvm-11-tools python3-pygments python3-yaml
Suggested packages:
  clang-11-doc llvm-11-doc python-pygments-doc ttf-bitstream-vera
The following NEW packages will be installed:
  binfmt-support clang-11 libclang-common-11-dev libclang-cpp11 libclang1-11 libffi-dev libllvm11
  libpfm4 libpocl2 libpocl2-common libz3-4 libz3-dev llvm-11 llvm-11-dev llvm-11-linker-tools
  llvm-11-runtime llvm-11-tools pocl-opencl-icd python3-pygments python3-yaml
0 upgraded, 20 new

In [3]:
import pyopencl as cl
import numpy
import numpy.linalg as la
import datetime
from time import time

In [5]:
a = numpy.random.rand(1000).astype(numpy.float32)
b = numpy.random.rand(1000).astype(numpy.float32)
c_result = numpy.empty_like(a)

# Speed in normal CPU usage
time1 = time()
for i in range(1000):
        for j in range(1000):
                c_result[i] = a[i] + b[i]
                c_result[i] = c_result[i] * (a[i] + b[i])
                c_result[i] = c_result[i] * (a[i] / 2.0)
time2 = time()
print ("Execution time of test without OpenCL: ", time2 - time1, "s");


Execution time of test without OpenCL:  2.9854466915130615 s


In [6]:
  for platform in cl.get_platforms():
    for device in platform.get_devices():
        print ("===============================================================");
        print ("Platform name:", platform.name);
        print ("Platform profile:", platform.profile);
        print ("Platform vendor:", platform.vendor);
        print ("Platform version:", platform.version);
        print ("---------------------------------------------------------------");
        print ("Device name:", device.name);
        print ("Device type:", cl.device_type.to_string(device.type));
        print ("Device memory: ", device.global_mem_size//1024//1024, 'MB');
        print ("Device max clock speed:", device.max_clock_frequency, 'MHz');
        print ("Device compute units:", device.max_compute_units);

        # Simnple speed test
        ctx = cl.Context([device])
        queue = cl.CommandQueue(ctx,
                properties=cl.command_queue_properties.PROFILING_ENABLE)

        mf = cl.mem_flags
        a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a)
        b_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b)
        dest_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)

        prg = cl.Program(ctx, """
            __kernel void sum(__global const float *a,
            __global const float *b, __global float *c)
            {
                        int loop;
                        int gid = get_global_id(0);
                        for(loop=0; loop<1000;loop++)
                        {
                                c[gid] = a[gid] + b[gid];
                                c[gid] = c[gid] * (a[gid] + b[gid]);
                                c[gid] = c[gid] * (a[gid] / 2.0);
                        }
                }
                """).build()

        exec_evt = prg.sum(queue, a.shape, None, a_buf, b_buf, dest_buf)
        exec_evt.wait()
        elapsed = 1e-9*(exec_evt.profile.end - exec_evt.profile.start)

        print ("Execution time of test: %g s" % elapsed);

        c = numpy.empty_like(a)
        cl._enqueue_read_buffer(queue, dest_buf, c).wait()
        error = 0
        for i in range(1000):
                if c[i] != c_result[i]:
                        error = 1
        if error:
                print ("Results doesn't match!!");
        else:
                print ("Results OK")

Platform name: NVIDIA CUDA
Platform profile: FULL_PROFILE
Platform vendor: NVIDIA Corporation
Platform version: OpenCL 3.0 CUDA 12.2.138
---------------------------------------------------------------
Device name: Tesla T4
Device type: ALL | GPU
Device memory:  15102 MB
Device max clock speed: 1590 MHz
Device compute units: 40
Execution time of test: 0.0012807 s
Results OK
Platform name: Portable Computing Language
Platform profile: FULL_PROFILE
Platform vendor: The pocl project
Platform version: OpenCL 2.0 pocl 1.8  Linux, None+Asserts, RELOC, LLVM 11.1.0, SLEEF, DISTRO, POCL_DEBUG
---------------------------------------------------------------
Device name: pthread-Intel(R) Xeon(R) CPU @ 2.20GHz
Device type: ALL | CPU
Device memory:  10930 MB
Device max clock speed: 2199 MHz
Device compute units: 2
Execution time of test: 0.0035631 s
Results OK
